In [5]:
import camb
import numpy as np
import os

def calculate_power_spectra(params, lensed=True):
    # Set up a new set of parameters for CAMB
    pars = camb.set_params(**params)
    # Calculate results for these parameters
    results = camb.get_results(pars)
    # Get dictionary of CAMB power spectra
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    # Choose the appropriate power spectrum
    if lensed:
        cl = powers['total']
    else:
        cl = powers['unlensed_scalar']
    # Cut out the first 2 elements of the power spectra, which are zero
    cl_cut = cl[2:]
    # Get the TT, TE, and EE power spectra
    cl_tt = cl_cut[:, 0]
    cl_te = cl_cut[:, 3]
    cl_ee = cl_cut[:, 1]
    return cl_tt, cl_te, cl_ee

params = {
    # Parameters that are part of the proposal distribution:
    'ombh2': 0.025,                # Baryon density parameter
    'omch2': 0.122,                # Cold dark matter density parameter
    'cosmomc_theta': 1.04109/100,       # Theta_MC - need to be transformed
    'tau': 0.06,                   # Optical depth to reionization
    'ns': 0.965,                   # Scalar spectral index
    'As': 2.0e-9,# the scalar amplitude, need to be transformed for proposal distribution

    # Fixed parameters:
    #'H0': 69.5,                    # Hubble constant in km/s/Mpc (Fixed)
    'mnu': 0.06,                   # Sum of neutrino masses in eV (Fixed)
    'omk': 0,                      # Curvature density parameter, flat universe (Fixed)
    'halofit_version': 'mead',     # Non-linear matter power spectrum model version (Fixed)
    'lmax': 2800                   # Maximum multipole number for calculations (Fixed)
}



cl_tt, cl_te, cl_ee = calculate_power_spectra(params, lensed=True)
print(cl_tt)
print(cl_te)
print(cl_ee)

[982.32562024 927.57538209 875.41964977 ...  36.22063115  36.11447954
  36.00921749]
[ 2.75474425  3.09469103  2.90353789 ... -1.26516941 -1.27129668
 -1.27752765]
[0.03586869 0.04609061 0.04003475 ... 1.56565523 1.56519372 1.56468134]


In [ ]:
import likelihood
from likelihood import PlanckLitePy

likelihood_function = likelihood.PlanckLitePy()

likelihood_function.loglike(cl_tt, cl_te, cl_ee)

-1058.7842310098813

In [7]:
def process_params_and_calculate_spectra(array):
    array = array.copy()  # Create a copy of the array
    # Modify the third and sixth elements of the array
    array[2] /= 100
    array[5] = np.exp(array[5]) / 1e10

    # Set the parameters for the proposal distribution
    params = {
        'ombh2': array[0],
        'omch2': array[1],
        'cosmomc_theta': array[2],
        'tau': array[3],
        'ns': array[4],
        'As': array[5],

        
        'mnu': 0.06,
        'omk': 0,
        'halofit_version': 'mead',
        'lmax': 2800
    }

    # Call the calculate_power_spectra function and return the result
    return calculate_power_spectra(params)

In [8]:
def proposal_distribution(current_state):
    step_sizes = np.array([
        np.sqrt(0.0001**2),  # ombh2
        np.sqrt(0.001**2),   # omch2
        np.sqrt(0.0004**2),  # theta_MC_100
        np.sqrt(0.006**2),   # tau
        np.sqrt(0.004**2),   # ns
        np.sqrt(0.001**2)    # log(10^10 As)
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step


def generate_initial_state():
    pass


def proposal_distribution_2(current_state):
    step_sizes = np.array([
        np.sqrt(0.0001**2),  # ombh2
        np.sqrt(0.001**2),   # omch2
        np.sqrt(0.0004**2),  # theta_MC_100
        np.sqrt(0.006**2),   # tau
        np.sqrt(0.004**2),   # ns
        np.sqrt(0.001**2)    # log(10^10 As)
    ])
    proposal_step = np.random.normal(0, step_sizes)
    return current_state + proposal_step

In [19]:
import numpy as np
import pickle 
import os
import time

class MCMC_MH:
    def __init__(self, likelihood, proposal_distribution, initial_state, num_samples, num_chains, stepsize=0.5, burnin_ratio=0.05, resume=False):
        self.likelihood = likelihood
        self.proposal_distribution = proposal_distribution
        self.curr_state = [initial_state]*num_chains
        self.curr_likeli = [likelihood.loglike(*process_params_and_calculate_spectra(initial_state))]*num_chains
        self.num_samples = num_samples
        self.num_chains = num_chains
        self.stepsize = stepsize
        self.burnin = int(burnin_ratio * num_samples)  # calculate burn-in steps as 5% of total samples
        self.samples = [[] for _ in range(num_chains)]
        self.checkpoint_interval = 5
        self.resume = resume

        if resume:
            self.load_checkpoint()
        elif os.path.exists('./checkpoints') and os.listdir('./checkpoints'):
            raise ValueError("The './checkpoints' directory should be empty when starting a new run.")

    def save_checkpoint(self, iteration, burn_in=False):
        if not os.path.exists('./checkpoints'):
            os.makedirs('./checkpoints')
        filename = f'./checkpoints/checkpoint_{"burnin" if burn_in else "production"}.pkl'
        with open(filename, 'wb') as f:
            pickle.dump((self.curr_state, self.samples), f)

    def load_checkpoint(self):
        checkpoint_files = [f for f in os.listdir('./checkpoints') if f.endswith('.pkl')]
        if not checkpoint_files:
            raise ValueError("No checkpoints found to resume from.")
        latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
        with open(f'./checkpoints/{latest_checkpoint}', 'rb') as f:
            self.curr_state, self.samples = pickle.load(f)

    def burn_in(self):
        start_time = time.time()
        if self.resume:
            self.load_checkpoint()

        for i in range(self.burnin):
            for j in range(self.num_chains):
                self.mcmc_updater(j)
            if (i + 1) % self.checkpoint_interval == 0:
                self.save_checkpoint(i + 1, burn_in=True)
                elapsed_time = time.time() - start_time
                print(f"Completed {i+1} burn-in steps in {elapsed_time} seconds")

    def mcmc_updater(self, chain_index):
        proposal_state = self.proposal_distribution(self.curr_state[chain_index])

        prop_loglikeli = self.likelihood.loglike(*process_params_and_calculate_spectra(proposal_state))
        print(prop_loglikeli)
        accept_crit = prop_loglikeli - self.curr_likeli[chain_index]
        accept_threshold = np.log(np.random.uniform(0, 1))

        if accept_crit > accept_threshold:
            print("Accepted")
            self.curr_state[chain_index], self.curr_likeli[chain_index] = proposal_state, prop_loglikeli
        else:
            print("Rejected")

        return self.curr_state[chain_index], self.curr_likeli[chain_index]

    def metropolis_hastings(self):
        start_time = time.time()
        self.burn_in()  # perform burn-in before production run

        if self.resume:
            self.load_checkpoint()

        for i in range(self.num_samples):
            for j in range(self.num_chains):
                self.curr_state[j], self.curr_likeli[j] = self.mcmc_updater(j)
                self.samples[j].append(self.curr_state[j])
            if (i + 1) % self.checkpoint_interval == 0:
                self.save_checkpoint(i + 1)
                elapsed_time = time.time() - start_time
                print(f"Completed {i+1} steps in {elapsed_time} seconds")
            if (i + 1) % 1 == 0:
                self.calculate_convergence()

        return self.samples

    def calculate_convergence(self):
        # Calculate the mean of each chain
        chain_means = [np.mean(chain) for chain in self.samples]

        # Calculate the variance of each chain
        chain_vars = [np.var(chain, ddof=1) for chain in self.samples]

        # Calculate the mean of all samples
        grand_mean = np.mean(chain_means)

        # Calculate B, the between-chain variance
        B = self.num_samples / (self.num_chains - 1) * np.sum((chain_means - grand_mean) ** 2)

        # Calculate W, the within-chain variance
        W = np.mean(chain_vars)

        # Calculate the variance estimate for the pooled chains
        var_estimate = (1 - 1 / self.num_samples) * W + 1 / self.num_samples * B

        # Calculate the potential scale reduction factor (R-1 statistic)
        R_minus_one = (var_estimate - W) / var_estimate

        print(f'R-1 statistic: {R_minus_one}')

In [20]:
initial_state = np.array([0.025, 0.122, 1.04109, 0.06, 0.965, np.log(2.0e-9*1e10)])

mcmc = MCMC_MH(likelihood_function, proposal_distribution, initial_state, 1000, stepsize=0.5, burnin_ratio=0.0, resume=False, num_chains=4)

In [21]:
mcmc.metropolis_hastings()

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'MCMC_MH' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/henrybae/miniconda3/envs/cosmology/lib/python3.9/multiprocessing/process.py", line 108,

KeyboardInterrupt: 